In [1]:
!pip install pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SalesDataAnalysis") \
    .getOrCreate()


In [3]:
product_path = "C:/Users/Gagan/Desktop/Projects DE/Project_5/product.csv"
sales_path = "C:/Users/Gagan/Desktop/Projects DE/Project_5/sales.csv"
customer_path = "C:/Users/Gagan/Desktop/Projects DE/Project_5/customer.csv"


In [4]:
products_df = spark.read.csv(product_path, header=True, inferSchema=True)
sales_df = spark.read.csv(sales_path, header=True, inferSchema=True)
customer_df = spark.read.csv(customer_path, header=True, inferSchema=True)


In [5]:
# Fill missing values with default values if necessary
products_df = products_df.na.fill({"Category": "Unknown"})
sales_df = sales_df.na.drop()
customer_df = customer_df.na.fill({"Age": 0})


In [19]:
from pyspark.sql.types import IntegerType, DoubleType

sales_df = sales_df.withColumn("Quantity", sales_df["Quantity"].cast(IntegerType()))
sales_df = sales_df.withColumn("Discount", sales_df["Discount"].cast(DoubleType()))


In [20]:
sales_products_df = sales_df.join(products_df, on="Product ID", how="inner")
full_df = sales_products_df.join(customer_df, on="Customer ID", how="inner")


In [8]:
full_df.show()

+-----------+---------------+----------+--------------+----------+----------+--------------+--------+--------+--------+--------+---------------+------------+--------------------+------------------+-----------+---+-------------+---------------+--------------+-----------+-------+
|Customer ID|     Product ID|Order Line|      Order ID|Order Date| Ship Date|     Ship Mode|   Sales|Quantity|Discount|  Profit|       Category|Sub-Category|        Product Name|     Customer Name|    Segment|Age|      Country|           City|         State|Postal Code| Region|
+-----------+---------------+----------+--------------+----------+----------+--------------+--------+--------+--------+--------+---------------+------------+--------------------+------------------+-----------+---+-------------+---------------+--------------+-----------+-------+
|   CG-12520|FUR-BO-10001798|         1|CA-2016-152156| 11/8/2016|11/11/2016|  Second Class|  261.96|       2|     0.0| 41.9136|      Furniture|   Bookcases|Bush S

In [9]:
#Total Sales for Each Product Category
full_df.groupBy("Category").sum("Sales").show()


+---------------+-----------------+
|       Category|       sum(Sales)|
+---------------+-----------------+
|Office Supplies|719047.0320000029|
|      Furniture|741999.7952999998|
|     Technology|836154.0329999966|
+---------------+-----------------+



In [10]:
#Average Discount Given Across All Products:
full_df.agg({"Discount": "avg"}).show()


+-------------------+
|      avg(Discount)|
+-------------------+
|0.15620272163298934|
+-------------------+



In [11]:
#Customer with the Highest Number of Purchases
full_df.groupBy("Customer ID").count().orderBy("count", ascending=False).show(1)


+-----------+-----+
|Customer ID|count|
+-----------+-----+
|   WB-21850|   37|
+-----------+-----+
only showing top 1 row



In [12]:
#Unique Products Sold in Each Region
full_df.groupBy("Region").agg({"Product ID": "count"}).distinct().show()


+-------+-----------------+
| Region|count(Product ID)|
+-------+-----------------+
|  South|             1626|
|Central|             2371|
|   East|             2762|
|   West|             3235|
+-------+-----------------+



In [13]:
#Total Profit Generated in Each State
full_df.groupBy("State").sum("Profit").show()


+--------------------+------------------+
|               State|       sum(Profit)|
+--------------------+------------------+
|                Utah|1818.1938000000005|
|           Minnesota| 7202.522500000001|
|                Ohio| 5985.887000000001|
|              Oregon|234.04950000000014|
|            Arkansas|          -62.9462|
|               Texas| 20528.91100000002|
|        Pennsylvania|13604.935000000007|
|         Connecticut| 533.4986999999999|
|            Nebraska|         1166.0176|
|              Nevada|278.06780000000003|
|          Washington|24405.796599999983|
|            Illinois| 9560.145599999993|
|            Oklahoma| 829.0181999999999|
|District of Columbia|490.95669999999996|
|            Delaware| 3336.382700000002|
|          New Mexico|         1340.1399|
|            Missouri|2212.8746999999994|
|        Rhode Island|         2276.7013|
|             Georgia|12781.342599999998|
|            Michigan|7752.2969000000085|
+--------------------+------------

In [14]:
#Product Sub-Category with the Highest Sales
full_df.groupBy("Sub-Category").sum("Sales").orderBy("sum(Sales)", ascending=False).show(10)


+------------+------------------+
|Sub-Category|        sum(Sales)|
+------------+------------------+
|      Phones| 330007.0540000001|
|      Chairs|328449.10300000076|
|     Storage|223843.60800000012|
|      Tables| 206965.5320000001|
|     Binders| 203412.7330000001|
|    Machines|189238.63099999996|
| Accessories| 167380.3180000001|
|     Copiers|149528.02999999994|
|   Bookcases|114879.99629999997|
|  Appliances|        107532.161|
+------------+------------------+
only showing top 10 rows



In [15]:
#Average Age of Customers in Each Segment
full_df.groupBy("Segment").avg("Age").show()


+-----------+------------------+
|    Segment|          avg(Age)|
+-----------+------------------+
|   Consumer| 44.60585628973223|
|Home Office| 43.28210880538418|
|  Corporate|44.816556291390725|
+-----------+------------------+



In [16]:
#Orders Shipped in Each Shipping Mode
full_df.groupBy("Ship Mode").count().show()


+--------------+-----+
|     Ship Mode|count|
+--------------+-----+
|   First Class| 1538|
|      Same Day|  543|
|  Second Class| 1945|
|Standard Class| 5968|
+--------------+-----+



In [17]:
#Total Quantity of Products Sold in Each City
full_df.groupBy("City").sum("Quantity").show()

+---------------+-------------+
|           City|sum(Quantity)|
+---------------+-------------+
|          Tyler|           22|
|    Springfield|          282|
|  Bowling Green|           89|
|         Auburn|          214|
|North Las Vegas|           29|
|        Phoenix|          256|
|  Lake Elsinore|           35|
|         Monroe|          184|
| Pembroke Pines|           48|
|       Westland|           54|
|    Lindenhurst|           42|
|         Marion|           77|
|          Omaha|           43|
|   Fort Collins|           55|
|        Everett|           24|
|     Greensboro|           51|
|       Franklin|          184|
|   Lincoln Park|           28|
|         Dallas|          602|
|    Thomasville|           40|
+---------------+-------------+
only showing top 20 rows



In [18]:
#Customer Segment with the Highest Profit Margin:
full_df.groupBy("Segment").sum("Profit").orderBy("sum(Profit)", ascending=False).show(1)


+--------+------------------+
| Segment|       sum(Profit)|
+--------+------------------+
|Consumer|134119.20919999972|
+--------+------------------+
only showing top 1 row

